In [ ]:
!unzip "/content/drive/MyDrive/IBM/Digital Naturalist Dataset.zip"

Archive:  /content/drive/MyDrive/IBM/Digital Naturalist Dataset.zip
   creating: Digital Naturalist Dataset/
   creating: Digital Naturalist Dataset/Bird/
   creating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (1).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (10).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (11).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (2).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (3).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (4).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (5).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great Indian Bustard Bird/download (6).jpg  
  inflating: Digital Naturalist Dataset/Bird/Great 

#Importing the required libraries:

In [ ]:
import os, gc, glob, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from PIL import Image
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

#Using the Inception V3 Pretrained model:

In [ ]:
base_model = InceptionV3(input_shape=(224,224,3),include_top=False)

#Freezing the weights in the pretrained model:

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

#Flatening and adding the dense layer:

In [ ]:
X = Flatten()(base_model.output)

#using sigmoid function as the activation funciton
X = Dense(units = 6, activation = 'sigmoid')(X)

#Compiling the model:

In [ ]:
model = Model(base_model.input,X)

#using adam as the optimizer
#accuracy is set as the metrics to judge the performance of the model
model.compile(optimizer = 'adam',loss = keras.losses.binary_crossentropy,metrics=['accuracy'])

#summarizing the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                             

#Loading the train and test data:

In [ ]:
#initializing the datagenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

#loading the train dataset from the directory
traindata= datagen.flow_from_directory(directory="/content/Digital Naturalist Dataset/Bird",target_size=(224,224),batch_size=100)

#loading the test dataset from the directory
testdata= datagen.flow_from_directory(directory="/content/Digital Naturalist Dataset/Mammal",target_size=(224,224),batch_size=100)

Found 42 images belonging to 2 classes.
Found 47 images belonging to 2 classes.


In [ ]:
traindata.class_indices

{'Great Indian Bustard Bird': 0, 'Spoon Billed Sandpiper Bird': 1}

#Model Checkpoint and Early Stopping:

In [ ]:
# Creating a model checkpoint which monitors the accuracy of the model and saves the checkpoint
mc = ModelCheckpoint(filepath = "./model.h5",
                     monitor = 'accuracy',
                     verbose = 1,
                     save_best_only = True)

# Creating a earlystopping object which stop training once the model performance stops improving on a hold out validation dataset
es = EarlyStopping(monitor = "accuracy",
                   min_delta = 0.01,
                   verbose = 1)

call_back = [mc,es]

#Model fitting

In [ ]:
# Fitting the model 
modelHistory = model.fit(traindata, steps_per_epoch=60,epochs = 30,callbacks=call_back,validation_data=testdata)

#Exporting the model

In [ ]:
# Exporting the model to json 
model_json = model.to_json()
with open("DigitalNaturalist.json", "w") as json_file:
    json_file.write(model_json)

# Exporting the model weights

model.save_weights("DigitalNaturalist")
print("Saved model to disk")

Saved model to disk


#Testing the model

In [ ]:
#Testing the model 
predictions = ["Corpse Flower", 
               "Great Indian Bustard", 
               "Lady's slipper orchid", 
               "Pangolin", 
               "Spoon Billed Sandpiper", 
               "Seneca White Deer"
              ]


path = '/content/Digital Naturalist Dataset/Mammal/Senenca White Deer Mammal/images (10).jpg'
ime = tf.keras.utils.load_img(path,target_size=(224,224))

i = tf.keras.preprocessing.image.img_to_array(ime)
i = preprocess_input(i)
input = np.array([i])
pred = model.predict(input)
predictions[np.argmax(pred)]

1/1 [==============================] - 0s 166ms/step


"Lady's slipper orchid"